In [1]:
import pandas as pd
from nltk import word_tokenize
# Remoção de acentos
from unicodedata import normalize
import _thread
import time
import requests
import re


# Tokenização com nltk
from nltk import word_tokenize

In [2]:
df = pd.read_csv('noticias_if.csv')
df = df.iloc[10:]

In [3]:
df = df[(df['descricao'].isnull() == False)]

In [4]:
import nltk

stopwords = nltk.corpus.stopwords.words('portuguese')
#print(stopwords)

In [37]:
from string import punctuation

punctuation+'“”–'

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~“”–'

In [38]:
def processa(row):
    txt = row['titulo'] + ' ' + row['descricao']
    
    txt = re.sub('<[^<]+?>', '', txt)
    
    return ' '.join([t for t in word_tokenize(txt.lower()) if (t not in stopwords) and (t not in punctuation)])

df['doc'] = df.apply(processa, axis=1)

df['doc'].head()

11    fapemig lança pacote editais 2014 fundação amp...
13    conselho pais realiza “ dia campo ” pais estud...
14    censoead.br 2012 2013 censoead.br 2012 objetiv...
17    integrantes seção cultura artes participam eve...
19    período inscrições sisu 2014 6 10 janeiro if s...
Name: doc, dtype: object

In [41]:
df.iloc[0].doc

'fapemig lança pacote editais 2014 fundação amparo pesquisa estado minas gerais fapemig lançou pacote editais ano 2014 destinando r 36 milhões recursos apoio pesquisadores mineiros lançamentos edital demanda universal provê apoio projetos qualquer área conhecimento demais modalidades abertas oferecem suporte trabalhos pesquisa desenvolvidos estado meio concessão bolsas pesquisadores financiamento itens apoio livros manutenção equipamentos confira seguir todos editais lançados especificidades universal edital 01/2014 apoia financeiramente projetos pesquisa científica tecnológica inovação diversas áreas conhecimento edital vai destinar todo r 23 milhões pesquisas aprovadas cada proposta deve ser r 50 mil propostas recebidas 31 março 2014. manutenção equipamentos edital 02/2014 financia reparos manutenção equipamentos laboratórios período garantia cujo custo aquisição sido superior r 100 mil todo distribuídos r 2 milhões propostas aprovadas propostas devem ser enviadas 2 abril programa pe

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1, 1),
    max_features=None,
    binary=False,
    use_idf=True
)
tfidf_matrix = vectorizer.fit_transform(df['doc'])

tfidf_matrix

<3080x18884 sparse matrix of type '<class 'numpy.float64'>'
	with 219300 stored elements in Compressed Sparse Row format>

In [10]:
pd.DataFrame(tfidf_matrix.todense(), columns=vectorizer.get_feature_names()).head()

00  000  000000  000080  0000ff  001  001087  002  002084  002086  ...    \
0  0.0  0.0     0.0     0.0     0.0  0.0     0.0  0.0     0.0     0.0  ...     
1  0.0  0.0     0.0     0.0     0.0  0.0     0.0  0.0     0.0     0.0  ...     
2  0.0  0.0     0.0     0.0     0.0  0.0     0.0  0.0     0.0     0.0  ...     
3  0.0  0.0     0.0     0.0     0.0  0.0     0.0  0.0     0.0     0.0  ...     
4  0.0  0.0     0.0     0.0     0.0  0.0     0.0  0.0     0.0     0.0  ...     

   último  últimos  única  únicas  único  únicos  úteis  útero  útil  útimo  
0     0.0      0.0    0.0     0.0    0.0     0.0    0.0    0.0   0.0    0.0  
1     0.0      0.0    0.0     0.0    0.0     0.0    0.0    0.0   0.0    0.0  
2     0.0      0.0    0.0     0.0    0.0     0.0    0.0    0.0   0.0    0.0  
3     0.0      0.0    0.0     0.0    0.0     0.0    0.0    0.0   0.0    0.0  
4     0.0      0.0    0.0     0.0    0.0     0.0    0.0    0.0   0.0    0.0  

[5 rows x 22706 columns]

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcula todas as similaridades de cada linha
sim = cosine_similarity(tfidf_matrix)

sim.shape

(3080, 3080)

In [52]:
sim[0][2]

0.011435244212165723

In [91]:
# Similaridade via distância de Levenshtein
from Levenshtein import distance

def trazer_mais_proximas(noticia):    
    
    l = []
    for i in range(0,len(df['doc'])):
        l.append(
        (
            distance(
            noticia,
            df.iloc[i].doc
            ), i)
        )
    l = sorted(l, key=lambda x: x[0])[0:5]
    [print(df.iloc[x[1]].link) for x in l]
    return [df.iloc[x[1]].link for x in l]

In [93]:
trazer_mais_proximas("professora Regina Lucia Pelachim Lianda, as estudantes do Curso de Licenciatura em Química, Izabela Azevedo Santos e Aline Aparecida Campos e o aluno do Curso Técnico em Química Integrado ao Ensino Médio, Vitor Roberto de Oliveira publicaram um artigo na revista Global Education da HAMK")

http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-sao-premiados-olimpiada-mineira-quimica-2012
http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-sao-premiados-olimpiada-mineira-quimica-omq-olimpiada-brasileira-q
http://www.barbacena.ifsudestemg.edu.br/destaques/coordenacao-geral-tecnico-integrado-divulga-oportunidade-bolsa-fazer-ano-ensino-medio-nos-
http://www.barbacena.ifsudestemg.edu.br/destaques/processo-seletivo-2014-divulgada-3%C2%AA-chamada-cursos-tecnicos-integrados-subsequentes-conc-0
http://www.barbacena.ifsudestemg.edu.br/destaques/processo-seletivo-2014-divulgada-2%C2%AA-chamada-cursos-tecnicos-integrados-subsequentes-concom


['http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-sao-premiados-olimpiada-mineira-quimica-2012',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/alunos-campus-barbacena-sao-premiados-olimpiada-mineira-quimica-omq-olimpiada-brasileira-q',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/coordenacao-geral-tecnico-integrado-divulga-oportunidade-bolsa-fazer-ano-ensino-medio-nos-',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/processo-seletivo-2014-divulgada-3%C2%AA-chamada-cursos-tecnicos-integrados-subsequentes-conc-0',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/processo-seletivo-2014-divulgada-2%C2%AA-chamada-cursos-tecnicos-integrados-subsequentes-concom']

In [97]:
trazer_mais_proximas("O Ministério da Saúde, a cada mês, faz campanhas direcionadas para uma situação específica de prevenção e promoção da saúde. Neste mês o foco principal é a prevenção ao suicídio. E para direcionar o assunto, aconteceu na noite de terça-feira da semana anterior para os alunos do 2º Período do Curso Técnico em Enfermagem, uma Palestra com o tema principal Valorização da Vida ministrada pelo Psicólogo Marcone Luiz de Melo. O palestrante falou sobre a questão do sofrimento mental e a necessidade de acolhimento, sensibilizando os alunos para o tema proposto., explicou a professora Estephania Suely G. Carvalho.")

http://www.barbacena.ifsudestemg.edu.br/destaques/campus-barbacena-realiza-palestra-valorizacao-vida
http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultados-selecoes-projetos-extensao-area-curso-tecnico-seguranca-trabalho
http://www.barbacena.ifsudestemg.edu.br/destaques/atencao-aviso-importante-funcionamento-refeitorio
http://www.barbacena.ifsudestemg.edu.br/destaques/centro-academico-licenciatura-quimica-agradece-apoio-pela-fundacao-ca
http://www.barbacena.ifsudestemg.edu.br/destaques/gestao-pessoas-campus-barbacena-recebe-projeto-%E2%80%9C-dedinho-prosa%E2%80%9D


['http://www.barbacena.ifsudestemg.edu.br/destaques/campus-barbacena-realiza-palestra-valorizacao-vida',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/divulgado-resultados-selecoes-projetos-extensao-area-curso-tecnico-seguranca-trabalho',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/atencao-aviso-importante-funcionamento-refeitorio',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/centro-academico-licenciatura-quimica-agradece-apoio-pela-fundacao-ca',
 'http://www.barbacena.ifsudestemg.edu.br/destaques/gestao-pessoas-campus-barbacena-recebe-projeto-%E2%80%9C-dedinho-prosa%E2%80%9D']

In [88]:
df.iloc[577].doc

'campus barbacena recebe grupo sudaneses realização curso capacitação'